# **5.3 Training an Exchange Correlation Functional using Deepchem**

In [8]:
!pip install --pre deepchem
# !pip install --upgrade git+https://github.com/diffqc/dqc.git  # Commented out due to h5py incompatibility with Python 3.12
!pip install pylibxc2

  Cloning https://github.com/diffqc/dqc.git to /tmp/pip-req-build-j80xurp8
  Running command git clone --filter=blob:none --quiet https://github.com/diffqc/dqc.git /tmp/pip-req-build-j80xurp8
  Resolved https://github.com/diffqc/dqc.git to commit 0fe821fc92cb3457fb14f6dff0c223641c514ddb
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Using cached basis_set_exchange-0.12-py3-none-any.whl.metadata (6.7 kB)
  Using cached h5py-3.1.0.tar.gz (371 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 2
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing backend dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install backend dependencies did not run successfull

슈뢰딩거 방적식 직접푸는 방식(Wavefunction) 원자가 조금만 많아져도 계산량 기하급수적으로 늘어남

DFT : 복잡한 파동함수 대신 전자밀도라는 3차원 정보만으로 에너지 계산

이때 전자들 사이의 복잡한 상호작용 항 XC Functional



In [ ]:
# Example: creating a DFTEntry object
from deepchem.feat.dft_data import DFTEntry
e_type= 'dm'
true_val= 'deepchem/data/tests/dftHF_output.npy'
systems = [{'moldesc': 'H 0.86625 0 0; F -0.86625 0 0','basis': '6-311++G(3df,3pd)'}]
dm_entry_for_HF = DFTEntry.create(e_type, true_val, systems)

#list of systems in an entry object
dm_entry_for_HF.get_systems()

dqc 라이브러리. DFT계산의 모든 과정이 미분가능하도록 설계

In [3]:
# INSTALL PYSCF
!pip install pyscf
import pyscf

In [5]:
from pyscf import gto, scf, cc
import numpy as np
import torch

# create a pyscf mol
mol = gto.M(atom= 'H 0.86625 0 0; F -0.86625 0 0', basis='6-311++G(3df,3pd)' ,unit="Bohr")
mf  = scf.UHF(mol).run()
mcc = cc.UCCSD(mf)
mcc.kernel()

# obtain the total density matrix
modm = mcc.make_rdm1()
aodm0 = np.dot(mf.mo_coeff[0], np.dot(modm[0], mf.mo_coeff[0].T))
aodm1 = np.dot(mf.mo_coeff[1], np.dot(modm[1], mf.mo_coeff[1].T))
aodm = aodm0 + aodm1

# save the value in a .npy file
dm = torch.as_tensor(aodm, dtype=torch.double)
outfile = "output.npy"
np.save(outfile, dm)

converged SCF energy = -100.057640174912  <S^2> = 1.459588e-10  2S+1 = 1
E(UCCSD) = -100.3581800968686  E_corr = -0.3005399219564535


CCSD(Coupled Cluster) : DFT보다 정밀, 계산량 많음, AI학습시키기위한 정답을 만들기 위해 사용

In [ ]:
import os
import requests
from deepchem.data.data_loader import DFTYamlLoader

# Define the directory and file name
assets_dir = 'assets'
file_name = 'dft_tutorial.yaml'
file_path = os.path.join(assets_dir, file_name)

# Create the assets directory if it doesn't exist
if not os.path.exists(assets_dir):
    os.makedirs(assets_dir)

# Download the yaml file if it doesn't exist locally
if not os.path.exists(file_path):
    print(f"Downloading {file_name}...")
    url = "https://raw.githubusercontent.com/deepchem/deepchem/master/examples/dft/assets/dft_tutorial.yaml"
    r = requests.get(url, allow_redirects=True)
    with open(file_path, 'wb') as f:
        f.write(r.content)
    print("Download complete.")

# Use the local file path
inputs = file_path

data = DFTYamlLoader()
dataset = data.create_dataset(inputs)

In [ ]:
# Example; using NNLDA layer
from deepchem.models.dft.nnxc import NNLDA
import torch
import torch.nn as nn
n_input, n_hidden = 2, 1
nnmodel = (nn.Linear(n_input, n_hidden))
output = NNLDA(nnmodel)

HybridXC&NNXC



NNLDA/NNGGA : 전통적인 물리 범함수(LDA,GGA)자리에 신경망 넣음

전통적인물리법칙 + 학습 가능한 신경망 결합한 형태

SCF

기존 DFT의 반복계산(SCF)과정은 미분이 불가능해 AI학습 어려움

DeepChem의 XCNNSCF는 과정 전체를 미분가능하도록 설


In [ ]:
# Example; calculating energy using the scf layer
from deepchem.models.dft.scf import XCNNSCF
import torch
from deepchem.feat.dft_data import DFTEntry, DFTSystem
from deepchem.models.dft.nnxc import HybridXC
nnmodel = (torch.nn.Sequential(torch.nn.Linear(2, 10),torch.nn.Tanh(),torch.nn.Linear(10, 1))).to(torch.double)
hybridxc = HybridXC("lda_x", nnmodel, aweight0=0.0)
evl = XCNNSCF(hybridxc, dm_entry_for_HF )
system = DFTSystem(systems[0])
run = evl.run(system)
output = run.energy()
print(output)

In [ ]:
# load the datasets
import numpy as np
inputs = 'assets/h2.yaml'
data = DFTYamlLoader()
h2_dataset = data.create_dataset(inputs)
predict = model.predict(h2_dataset)

In [ ]:
import matplotlib.pyplot as plt
#plot
true_val = (h2_dataset.y)
predict_val = (predict)*(-1)
pos = np.array([0.5000, 0.6842, 0.8684, 1.0526, 1.2368,
1.4211,
1.6053,
1.7895,
1.9737,
2.1579,
2.3421,
2.5263,
2.7105,
2.8974,
3.0789,
3.2632,
3.4474,
3.6316,
3.8158,
4.000])
plt.plot(pos, true_val )
plt.plot(pos, predict_val, color = 'orange')
y = pos * 0
plt.xlabel("H-H distance in Angstrom")
plt.ylabel("Energy in Hartree")
plt.xlim([0.0, 4.5])
plt.ylim([-0.2, 0.55])
plt.grid()
plt.show()

💡Insight

DQC프레임 워크 사용 신경망이 DFT 내에서 XC functional학습하도록 설계

Hybrid XC를 통해 물리적 제약 유지하면서 데이터 기반의 물성 예측 성능 확보

CCSD급의 고정밀 데이터를 학습 타겟으로 설정해 시뮬레이션을 AI모델로 대체 가능